In [ ]:
!pip install "tensorflow==1.15"

In [ ]:
!pip install "tensorflow_hub==0.6.0"

In [ ]:
pip install simpletransformers

In [ ]:
pip install h5py==2.10.0

In [ ]:
# load packages
from sklearn import metrics,preprocessing,model_selection
from sklearn.metrics import accuracy_score
import tensorflow.keras
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import pandas as pd
import re
import spacy
import nltk

from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en import English
spacy.load('en')
parser = English()
from tqdm import tqdm

In [ ]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_hub as hub

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Stop words and special characters
STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”","''"]

In [ ]:
def tokenizeText(text):

    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()

    tokens = parser(text)

    # lemmatization
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # reomve stop words and special charaters
    tokens = [tok for tok in tokens if tok.lower() not in STOPLIST]
    #tokens = [tok for tok in tokens if tok not in SYMBOLS]

    tokens = [tok for tok in tokens if len(tok) >= 3]

    # remove remaining tokens that are not alphabetic
    tokens = [tok for tok in tokens if tok.isalpha()]

    tokens = list(set(tokens))

    return ' '.join(tokens[:])

In [ ]:
def encode(le_enc, labels):
    enc = le_enc.transform(labels)
    return tensorflow.keras.utils.to_categorical(enc)

def decode(le_enc, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le_enc.inverse_transform(dec)

In [ ]:
df_train = pd.read_csv('rephrase_train.csv',usecols=['text','sarcastic'],encoding='utf8')

In [ ]:
df_train.shape

(1734, 2)

In [ ]:
df_train['sarcastic'].unique()

array([1, 0])

In [ ]:
df_train = df_train.dropna()
df_train['text'] = df_train['text'].apply(lambda x:tokenizeText(x))

In [ ]:
# ELMo Embedding
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [ ]:
x = df_train['text'].tolist()
y = df_train['sarcastic'].tolist()

le_enc = preprocessing.LabelEncoder()
le_enc.fit(y)

y_en = encode(le_enc, y)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(np.asarray(x), np.asarray(y_en), test_size=0.2, random_state=42)

In [ ]:
# Build Model
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=10, batch_size=16)
    model.save_weights('./response-elmo-model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./response-elmo-model.h5')
    predicts = model.predict(x_test, batch_size=16)
    #predictions = model.predict(df)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



2022-01-31 16:42:51.628882: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-01-31 16:42:51.644329: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-31 16:42:51.644421: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a93ffaa35db3): /proc/driver/nvidia/version does not exist
2022-01-31 16:42:51.645996: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


2022-01-31 16:42:51.661245: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200150000 Hz
2022-01-31 16:42:51.662428: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555accd3ea00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-01-31 16:42:51.662475: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version



Train on 1387 samples
Epoch 1/10
1387/1387 [==============================] - 230s 166ms/sample - loss: 0.6917 - acc: 0.5984
Epoch 2/10
1387/1387 [==============================] - 226s 163ms/sample - loss: 0.5855 - acc: 0.7015
Epoch 3/10
1387/1387 [==============================] - 223s 161ms/sample - loss: 0.5543 - acc: 0.7260
Epoch 4/10
1387/1387 [==============================] - 231s 166ms/sample - loss: 0.5059 - acc: 0.7628
Epoch 5/10
1387/1387 [==============================] - 223s 161ms/sample - loss: 0.4605 - acc: 0.7815
Epoch 6/10
1387/1387 [==============================] - 227s 163ms/sample - loss: 0.4111 - acc: 0.8255
Epoch 7/10
1387/1387 [==============================] - 226s 163ms/sample - loss: 0.3691 - acc: 0.8464
Epoch 8/10
1387/1387 [==============================] - 235s 169ms/sample - loss: 0.3663 - acc: 0.8443
Epoch 9/10
1387/1387 [==============================] - 231s 166ms/sample - loss: 0.2869 - acc: 0.8933
Epoch 10/10
1387/1387 [============================

In [ ]:
# decode test labels
y_test = decode(le_enc, y_test)
# decode predicted labels
y_preds = decode(le_enc, predicts)

In [ ]:
print(y_test)

[1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1
 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1
 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 1 0 1
 1 0 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 1 0 1 0
 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0
 1 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0
 1 1 0 0 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0
 1 1 0 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 1 0 1 0 0 0
 1 0 1 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0
 0 1 0 0 1 0 0 1 1 1 1 0 1 1]


In [ ]:
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.59      0.59      0.59       164
           1       0.63      0.63      0.63       183

    accuracy                           0.61       347
   macro avg       0.61      0.61      0.61       347
weighted avg       0.61      0.61      0.61       347



In [ ]:
print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))

Accuracy of ELMO is: 0.6109510086455331


**TESTING**

In [ ]:
#upload test1.csv
from google.colab import files
uploaded = files.upload()

Saving taskC.En.input.csv to taskC.En.input.csv


In [ ]:
df_test1 = pd.read_csv('taskC.En.input.csv',usecols=['text_0','text_1'],encoding='utf8')
df_test2 = pd.read_csv('taskC.En.input.csv',usecols=['text_1'],encoding='utf8')

In [ ]:
df1 = np.array(df_test1['text_1'])
df2 = np.array(df_test1['text_0'])

In [ ]:
df_test1 = df_test1.dropna()
#df2 = df1.dropna()
df_test1['text_1'] = df_test1['text_1'].apply(lambda x:tokenizeText(x))
df_test1['text_0'] = df_test1['text_0'].apply(lambda x:tokenizeText(x))

In [ ]:
x1 = df_test1['text_1'].tolist()
x2 = df_test1['text_0'].tolist()
x_test0 = np.asarray(x2)
x_test1 = np.asarray(x1)

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./response-elmo-model.h5')
    rp0 = model.predict(x_test0, batch_size=16)

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./response-elmo-model.h5')
    rp1 = model.predict(x_test1, batch_size=16)

In [ ]:
rp0 = rp0.round()
rp1 = rp1.round()
#print(rp)

In [ ]:
arr0 = []
arr1 = []
for i in rp0:
  arr0.append(int(i[1]))
for i in rp1:
  arr1.append(int(i[1]))

In [ ]:
print(arr0)
print("\n")
print(arr1)

[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1]


[0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 

In [ ]:
result = []
print(len(arr0), len(arr1))
for i in range(len(arr0)):
  #print(i)
  if i>=len(arr0):
    #print(i)
    break
  if(arr0[i]>arr1[i]):
    result.append(1)
  else: result.append(0)

200 200


In [ ]:
print(result)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]


In [ ]:
np.savetxt("GetSmartMSEC_c_en.txt", result, fmt="%s")

In [ ]:
np.savetxt("pred1.txt", arr1, fmt="%s")

In [ ]:
# decode predicted labels
x1_preds = decode(le_enc, predicts)

In [ ]:
f1 = open("predictions.txt",'w+')
f1.write(str(result))

600